In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Correctly point to the .env file
env_path = Path(os.getcwd()) / 'src' / '.env'

# Debugging step to verify the path
print(f"Checking if .env exists at: {env_path}")
print(f"Exists: {env_path.exists()}")

# Load the .env file
load_dotenv(env_path)

# Check if the variable is in the environment
if 'HOPSWORKS_API_KEY' in os.environ:
    print("HOPSWORKS_API_KEY is set.")
else:
    print("HOPSWORKS_API_KEY is not found in environment.")




Checking if .env exists at: /Users/farshid/taxi_demand_predictor/notebooks/src/.env
Exists: False
HOPSWORKS_API_KEY is not found in environment.


In [11]:
import os
from dotenv import load_dotenv
from pathlib import Path

# Dynamically find the root of the project
project_root = Path(__file__).resolve().parent.parent  # Adjust path relative to this script
env_path = project_root / 'src' / '.env'

# Debugging step to verify the path
print(f"Checking if .env exists at: {env_path}")
print(f"Exists: {env_path.exists()}")

# Load the .env file
load_dotenv(env_path)

# Check if the variable is in the environment
if 'HOPSWORKS_API_KEY' in os.environ:
    print("HOPSWORKS_API_KEY is set.")
else:
    print("HOPSWORKS_API_KEY is not found in environment.")


NameError: name '__file__' is not defined

In [ ]:
from datetime import datetime
import pandas as pd
from src.data import load_raw_data

from_year = 2022
to_year = datetime.now().year
print(f'Downloading raw data from {from_year} to {to_year}')

rides = pd.DataFrame()
for year in range(from_year, to_year+1):
    
    # download data for the whole year
    rides_one_year = load_raw_data(year)
    
    # append rows
    rides = pd.concat([rides, rides_one_year])

In [ ]:
print(f'{len(rides)=:,}')


In [ ]:
from src.data import transform_raw_data_into_ts_data

ts_data = transform_raw_data_into_ts_data(rides)

In [33]:
# string to datetime
ts_data['pickup_hour'] = pd.to_datetime(ts_data['pickup_hour'], utc=True)

# add column with Unix epoch milliseconds
ts_data['pickup_ts'] = ts_data['pickup_hour'].astype(int) // 10**6

In [34]:
import hopsworks


In [ ]:
from dotenv import load_dotenv
load_dotenv()


In [ ]:
# connect to the project
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)